# First, let's read the celebrity name CSVs 

In [1]:
# Let us install the library we are going to use to download images:
! pip install simple_image_download
! pip install scikit-image

INFO: pip is looking at multiple versions of simple-image-download to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 28.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 kB 24.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import boto3
import json
import io
import cv2
import skimage

import numpy as np
from IPython.display import Image as IImage
from skimage.exposure import is_low_contrast

from PIL import Image, ImageDraw, ExifTags, ImageColor
from matplotlib.pyplot import imshow


In [4]:
# Please download these CSV files from the github 

# Link to the github: https://github.com/sophiaxxiao/Celebrity_recognition_project.git

# You can type in the terminal 'git clone https://github.com/sophiaxxiao/Celebrity_recognition_project.git' to obtain these files


# Make sure you have the correct paths 
chinese_celebs = pd.read_csv('/home/ec2-user/SageMaker/Celebrity_recognition_project/chinese_celebs.csv')

western_celebs = pd.read_csv('/home/ec2-user/SageMaker/Celebrity_recognition_project/western_celebs.csv')


In [5]:
chinese_celebs

,Rank,Name,Earnings,Occupation,Country
0,1,Fan Bingbing,300 million yuan,Actress,China
1,2,Lu Han,210 million yuan,Actor and Singer,China
2,3,Yang Mi,200 million yuan,Actress,China
3,4,Zhao Liying,190 million yuan,Actress,China
4,5,Yang Yang,240 million yuan,Actor,China
...,...,...,...,...,...
73,75,Xie Na,40 million yuan,TV Host and Actress,China
74,76,Hawick Lau,75 million yuan,Actor,China
75,78,Huang Xuan,50 million yuan,Actor,China
76,79,Ma Sichun,40 million yuan,Actress,China


In [6]:
western_celebs

,Rank,Name,Earnings,Occupation,Country
0,1,Diddy,$130 million,Musician,U.S.
1,2,Beyoncé,$105 million,Musician,U.S.
2,3,J.K. Rowling,$95 million,Author,U.K.
3,4,Drake,$94 million,Musician,Canada
4,6,The Weeknd,$92 million,Musician,Canada
...,...,...,...,...,...
75,95,Chance The Rapper,$33 million,Musician,U.S.
76,95,Katy Perry,$33 million,Musician,U.S.
77,97,Carmelo Anthony,$32.6 million,Athlete,U.S.
78,98,Jason Aldean,$32.5 million,Musician,U.S.


In [ ]:
from simple_image_download import simple_image_download as simp 

chinese_celebrity_names = chinese_celebs['Name'][0:25] # selecting the top 25 celebrities 

response = simp.simple_image_download

# The code below downloads the images
for name in chinese_celebrity_names:
 response().download(name , 50)

HTTPSConnectionPool(host='pcwww.liv.ac.uk', port=443): Max retries exceeded with url: /~hanlu/LuHan2021.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))
HTTPSConnectionPool(host='elle.com.sg', port=443): Max retries exceeded with url: /wp-content/uploads/2022/05/Yang-Mi_1200x500-1.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f89c1824ac0>: Failed to establish a new connection: [Errno -2] Name or service not known'))
HTTPSConnectionPool(host='www.easternkicks.com', port=443): Max retries exceeded with url: /wp-content/uploads/2016/01/1_Jackie-Chan-as-Huo-An_3-1272x677.jpg (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1007)')))
HTTPSConnectionPool(host='thelosangelestribune.com', port=443): Max retries exceeded with url: /wp-content/uploads/2023/04/Jay-Chou-202

In [11]:
chinese_celeb_bucket = 'chinese-celebs' # (you need to change the name of the bucket -- make sure your bucket name is unique)

# Use this commmand to make the bucket 
!aws s3 mb s3://$chinese_celeb_bucket

# This is the AWS CLI command to create an S3 bucket. 
# The mb stands for "make bucket," and it's used to create a new S3 bucket.
# 's3://asian-faces-bucket' indicates that we're making our bucket in s3, and the part after the '//' is the name of the bucket


make_bucket: chinese-celebs


In [12]:
foldername = 'simple_images' # make sure you change this string to the name of the folder where your images are located 

!ls $foldername 
!aws s3 mv ./$foldername s3://$chinese_celeb_bucket/ --recursive # move the contents of the folder into the s3 bucket


Andy Lau face	     Huang Zitao face	Liu Shishi face     Wallace Huo face
Angelababy face      Hu Ge face		Liu Tao face	    Wang Kai face
Chen Kun face	     Jackie Chan face	Liu Yifei face	    William Chan face
Chen Xuedong face    Jay Chou face	Li Yifeng face	    Wu Xiubo face
Chris Lee face	     Jia Nailiang face	Lu Han face	    Xue Zhiqian face
Deng Chao face	     Jin Dong face	Lu Yi face	    Yang Mi face
Dilraba face	     Jing Boran face	Mark Chao face	    Yang Yang face
Eddie Peng face      Jing Tian face	Nicky Wu face	    Zhang Yixing face
Fan Bingbing face    Joe Chen face	Shu Qi face	    Zhao Liying face
Faye Wong face	     Kris Wu face	Sun Li face	    Zhao Wei face
He Jiong face	     Li Bingbing face	Tang Yan face	    Zheng Shuang face
Huang Bo face	     Lin Chi-Ling face	Tong Liya face
Huang Xiaoming face  Lin Gengxin face	Wallace Chung face
move: simple_images/Andy Lau face/Andy Lau face_1.jpg to s3://chinese-celebs/Andy Lau face/Andy Lau face_1.jpg
move: simple_images/Andy L

In [13]:
client=boto3.client('rekognition')
import subprocess
import re
import s3fs 

In [14]:
# this function checks if the image is low contrast

def low_contrast_check(photo):
    image = cv2.imread(photo)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return is_low_contrast(gray, fraction_threshold=0.35)

%matplotlib inline


In [19]:
import boto3
from PIL import Image
from io import BytesIO

# Initialize the S3 client
s3 = boto3.client('s3')

bucket_name = 'chinese-celebs'

# List objects in the bucket
response = s3.list_objects_v2(Bucket=bucket_name)

# Initialize Rekognition client
rekognition_client = boto3.client('rekognition')

# Function to remove photo based on your criteria
def should_remove_photo(photo, bucket):
    # Perform label detection using Rekognition
    try:
        response = rekognition_client.detect_faces(Image={'S3Object': {'Bucket': bucket, 'Name': photo}})
    except rekognition_client.exceptions.InvalidImageFormatException as e:
        print(f"Invalid image format: {e}")
        return True
    print(response)
    # Perform image checks
    if response['FaceDetails'] == []:
        return True
    
    image = Image.open(BytesIO(s3.get_object(Bucket=bucket, Key=photo)['Body'].read()))
    
    imgWidth, imgHeight = image.size

    face_details = response['FaceDetails'][0] if 'FaceDetails' in response and response['FaceDetails'] else None
    pitch = face_details['Pose']['Pitch'] if 'Pose' in face_details and 'Pitch' in face_details['Pose'] else None
    yaw = face_details['Pose']['Yaw'] if 'Pose' in face_details and 'Yaw' in face_details['Pose'] else None
    sunglasses = face_details.get('Sunglasses', {}).get('Value', None)
    bounding_box = face_details.get('BoundingBox', {})
    width = imgWidth * bounding_box.get('Width', 0)
    height = imgHeight * bounding_box.get('Height', 0)
    eyes_open = face_details.get('EyesOpen', {}).get('Value', None)
    
    # Check if Emotions array is present and non-empty
    emotions = face_details.get('Emotions', [])
    emotion = emotions[0]['Type'] if emotions else None
    emotion_confidence = emotions[0]['Confidence'] if emotions else None
    
    mouth_open = face_details.get('MouthOpen', {}).get('Value', None)

    
    if width < 50 or height < 50:
        print('Too small')
        return True
    if abs(yaw) > 45:
        print('yaw too much')
        return True
    if pitch > 45 or pitch < -30:
        print('pitch too severe')
        return True
    if not eyes_open or sunglasses or mouthOpen:
        print('eye, sunglasses, or mouth')
        return True
    return False

# Iterate through the objects in the bucket
for obj in response.get('Contents', []):
    # Get the object key (file path)
    object_key = obj['Key']
    print(object_key)

    # Apply your function to the object
    if should_remove_photo(object_key, bucket_name):
        print("Removing photo:", object_key)

        # Remove the file from the bucket
        s3.delete_object(Bucket=bucket_name, Key=object_key)
        
        print("Photo removed.")
    else:
        print("Keeping photo:", object_key)


Andy Lau face/Andy Lau face_17.jpg
Invalid image format: An error occurred (InvalidImageFormatException) when calling the DetectFaces operation: Request has invalid image format
Removing photo: Andy Lau face/Andy Lau face_17.jpg
Photo removed.
Andy Lau face/Andy Lau face_18.jpg
{'FaceDetails': [{'BoundingBox': {'Width': 0.4093947112560272, 'Height': 0.7138468623161316, 'Left': 0.13888609409332275, 'Top': 0.11697828024625778}, 'Landmarks': [{'Type': 'eyeLeft', 'X': 0.19889268279075623, 'Y': 0.417972207069397}, {'Type': 'eyeRight', 'X': 0.3494337797164917, 'Y': 0.37926581501960754}, {'Type': 'mouthLeft', 'X': 0.2401498258113861, 'Y': 0.6806560754776001}, {'Type': 'mouthRight', 'X': 0.36548149585723877, 'Y': 0.6495383977890015}, {'Type': 'nose', 'X': 0.23244930803775787, 'Y': 0.5493308305740356}], 'Pose': {'Roll': -11.013501167297363, 'Yaw': -27.373001098632812, 'Pitch': 1.1562918424606323}, 'Quality': {'Brightness': 78.89761352539062, 'Sharpness': 67.22731018066406}, 'Confidence': 99.999

InvalidS3ObjectException: An error occurred (InvalidS3ObjectException) when calling the DetectFaces operation: Request has invalid S3 object